In [1]:
# Cài đặt thư viện
%pip install paddleocr paddlepaddle layoutparser[layoutmodels,tesseract] -q
%pip install pandas transformers underthesea -q



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.__version__)

2.7.0+cpu


In [3]:
import layoutparser as lp
import detectron2

model = lp.Detectron2LayoutModel(
    config_path='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5]
)

ModuleNotFoundError: No module named 'detectron2'

In [4]:
# Import thư viện
import layoutparser as lp
import cv2, json, re
import pandas as pd
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


In [5]:
# Khởi tạo OCR tiếng Việt
ocr = PaddleOCR(use_angle_cls=True, lang='vi')


[2025/05/08 11:28:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\doman/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\doman/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48,

In [6]:
# Đọc ảnh hợp đồng khách sạn và phân tích bố cục
image_path = 'images/page_0.png'  # ← Đổi tên ảnh tại đây
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

model = lp.Detectron2LayoutModel('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config')
layout = model.detect(image_rgb)

# Vẽ các khối văn bản
layout.draw(image_rgb, box_width=3)
plt.figure(figsize=(12, 10))
plt.imshow(image_rgb)
plt.axis('off')
plt.show()


AttributeError: module layoutparser has no attribute Detectron2LayoutModel

In [ ]:
# OCR từng block văn bản theo layout
all_text = []
for block in layout:
    x_1, y_1, x_2, y_2 = map(int, block.coordinates)
    cropped = image[y_1:y_2, x_1:x_2]
    result = ocr.ocr(cropped, cls=True)
    text = ' '.join([line[1][0] for line in result[0]])
    all_text.append(text)

merged_text = ' '.join(all_text)
print(merged_text)


✅ Cách 1: Trích xuất bằng Regex

In [ ]:
def extract_with_regex(text):
    info = {}
    patterns = {
        "hotel_name": r"Khách sạn ([A-Z\w\s]+)",
        "address": r"Địa chỉ: ([^\n]+)",
        "contract_number": r"Số hợp đồng: ([\w\d/-]+)",
        "start_date": r"(\d{1,2}/\d{1,2}/\d{4})",
        "end_date": r"đến ngày (\d{1,2}/\d{1,2}/\d{4})",
        "total_value": r"Tổng giá trị(?: hợp đồng)?: ([\d.,]+ VNĐ)"
    }
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            info[key] = match.group(1).strip()
    return info

info_regex = extract_with_regex(merged_text)
print("===> Kết quả từ Regex:")
print(json.dumps(info_regex, indent=2, ensure_ascii=False))
pd.DataFrame([info_regex])


🤖 Cách 2: Trích xuất bằng NER (VinBERT hoặc tương đương)

In [ ]:
# Tải mô hình NER tiếng Việt
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vibert4news-base-cased")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vibert4news-base-cased")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


In [ ]:
# Chạy NER
ner_results = ner_pipeline(merged_text)
entities = {}
for r in ner_results:
    label = r['entity_group']
    text = r['word']
    if label not in entities:
        entities[label] = []
    entities[label].append(text)

info_ner = {k: ' '.join(v) for k, v in entities.items()}
print("===> Kết quả từ NER model:")
print(json.dumps(info_ner, indent=2, ensure_ascii=False))
pd.DataFrame([info_ner])
